In [1]:
# libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
    
import re

from tqdm import tqdm
import time

import sqlite3

import os
import random
import string

In [2]:
# PANDAS OPTIONS
# Set maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set the maximum column width to a high value
pd.set_option('display.max_colwidth', 1000)

### data import

In [18]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [19]:
df.head(2)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4


### new way 1

In [6]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ 1st

def find_ammo_quantity(text):
    pattern = r'\b(50|525|325)\b'  # Regular expression pattern to match required numbers
    match = re.search(pattern, text)
    return int(match.group()) if match else 0

def process_first_match(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_ammo_quantity)
    df['ammo_qtty_description'] = df['product_description'].apply(find_ammo_quantity)
    return df

df_v1 = process_first_match(df)

In [8]:
df_v1.sample(5)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description
1072,Balles Norma Biath Match - Cal. 22LR 22LR Par 20 40,https://www.naturabuy.fr/Balles-Norma-Biath-Match-Cal-22LR-22LR-Par-20-40-item-10290357.html,Norma,1,359.10,6.90,"Marque : NormaEtat de l'objet : NeufType : MatchType d'ogive : Plomb Calibre : 22LRQuantité : Par 20Grains : 40 Cartouche de qualité compétition avec un nouveau standard de précision pour les tireurs de biathlon de classe mondiale Caractéristiques : - Précision et fiabilité inégalées grâce à un processus de production affiné - Chaque lot est inspecté et testé pour s'assurer qu'il répond aux normes les plus élevées - Vitesse uniforme Afin de produire les meilleurs résultats pour le tir de biathlon - cartouches ont été développées en chambre froide (jusqu'à -20 ° C) - Calibre : 22LR - Ogive : Biath Match - Quantité : par 1, par 10 ou par 20 (voir options) Calibre 22LR Grammes 2.6 g Grains 40 Vitesse à 0 m 320 m/s Vitesse à 100 m 275 m/s Vitesse à 200 m 244 m/s Energie à 0 m 137 joules Energie à 100 m 101 joules Energie à 200 m 79 joules Energie à 300 m 61 joules L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et v...",366.00,0,0
859,500 Cartouches à balle Geco 22LR semi-auto,https://www.naturabuy.fr/500-Cartouches-balle-Geco-22LR-semi-auto-item-10667518.html,Geco,1,50.80,16.80,"Marque : GecoEtat de l'objet : NeufType : StandardType d'ogive : Cuivre AP5-Pack : 7872-defaultCombinationCoté technique: Geco 22 LR 10 Boites de 50 munitions 2.60g 350m/s Ceux qui recherchent des munitions à percussion annulaire performantes mais abordables sur le plan financier, trouveront leur bonheur chez GECO: fonctionnement parfait, régularité et précision, trois critères sur lesquels vous pourrez compter.La cartouche GECO Semi-Auto constitue une munition de qualité garantissant la précision nécessaire à un prix abordable. VITESSE 0m50m100mV [m/s] 350 306 279 ÉNERGIE 0m50m100mE [J] 159 122 101 50m100m150m200m250m300mDRO 50m ? -21,1 Arme et munitions de catégorie C, merci de nous fournir les documents suivants :- Pièce d'identité recto/verso (CNI ou passeport) + Justificatif de domicile (- de 3 mois)- Permis de chasser + validation du permis de chasser de l'année en cours ou licence de Tir FFBT ou FFTir en cours de validité signée et tamponnée par le médecin.Détails techniques...",67.60,0,50
589,Balles 22lr Subsonic Winchester® 42 grains,https://www.naturabuy.fr/Balles-22lr-Subsonic-Winchester-42-grains-item-4233440.html,Winchester,1,11.99,6.99,"Marque : WinchesterEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Pour réduire le bruit et gagner en discrétion, optez pour les balles 22LR Subsonic !La marque emblématique Winchester® propose une large sélection de produits convenant à de nombreuses situations. Essayez les munitions 22LR Subsonic ! Ces balles sont silencieuses grâce à leur vitesse subsonique, mais surtout, elles sont efficaces grâce à leur expansion maximum et un important transfert dénergie.Caractéristiques du produit : - Boite de 50 balles- Charge : 42 grains- Poids : 2.72g- Vitesse : 325m/s- Spécial nuisible- Pointe creuse tronquée en plomb. - Catégorie d'arme : Catégorie C",18.98,0,50
1115,MUNITIONS RWS RIFLE MATCH S Cal.22LR 40gr 2.6g,https://www.naturabuy.fr/MUNITIONS-RWS-RIFLE-MATCH-S-Cal-22LR-40gr-2-6g-item-10735905.html,RWS,1,7.50,14.90,Marque : RWSEtat de l'objet : NeufType : MatchType d'ogive : Plomb MUNITIONS RWS RIFLE MATCH S Cal.22LR 40gr 2.6g Vitesse proche du supersonique (345 m/s) Vendu par boite de 50 munitions,22.40,0,50
1063,Cartouches Winchester Wildcat Dynapoint - Cal. 22LR Par 1 22LR 40,https://www.naturabuy.fr/Cartouches-Winchester-Wildcat-Dynapoint-Cal-22LR-Par-1-22LR-40-item-10291446.html,Winchester,1,67.10,6.90,"Marque : WinchesterEtat de l'objet : NeufType : StandardType d'ogive : Plomb Quantité : Par 1Calibre : 22LRGrains : 40 

### new way 2

In [9]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ last

def find_all_ammo_quantities(text):
    pattern = r'\b(50|525|325)\b'  # Regular expression pattern to match required numbers
    matches = re.findall(pattern, text)
    return [int(match) for match in matches]

def process_all_matches(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_all_ammo_quantities)
    df['ammo_qtty_description'] = df['product_description'].apply(find_all_ammo_quantities)
    return df

df_v2 = process_all_matches(df)

In [10]:
df_v2.sample(5)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description
127,Cartouches Winchester M22 Lead Round Nose - 22LR - Par 10 / 22LR,https://www.naturabuy.fr/Cartouches-Winchester-M22-Lead-Round-Nose-22LR-Par-10-22LR-item-10757343.html,NaN,1,139.50,9.90,"Etat de l'objet : Neuf Les munitions M22 sont dotées d'une ogive cuivrée à tête ronde d'un poids de 40 grains (black copper plated round nose). Cette balle est à la fois précise et fiable dans les chargeurs haute capacité. De plus, son revêtement réduit l'encrassement de votre canon tout en autorisant de hautes vitesses. Caractéristiques : - Boîte de 400- Poids : 40 gr- Calibre 22 LR- Type d'ogive : Plomb Quantité: Par 1, par 10 ou par 20 (voir options)",149.40,[],[]
254,Boîte de 50 Munitions Fiocchi Target Sport 40 Grains - Cal. 22LR - 22LR / Par 10 / 40,https://www.naturabuy.fr/Boite-50-Munitions-Fiocchi-Target-Sport-40-Grains-Cal-22LR-Par-10-item-9207358.html,Fiocchi,1,125.10,9.90,"Marque : FiocchiEtat de l'objet : NeufType : StandardType d'ogive : Plomb Boîte de 50 munitions Fiocchi 22LR Target Sport Caractéristiques: - Grains : 40 grs- Boite de 50- Calibre : 22 LR Quantité: Par 1, par 10 ou par 20 (voir options)",135.00,[50],"[50, 50]"
882,( 22Lr Eley High Velocity HP)Munitions ELEY High Velocity Hollow Point cal. 22 LR,https://www.naturabuy.fr/-Munitions-ELEY-High-Velocity-Hollow-Point-cal-22-LR--item-5341814.html,Eley,1,106.48,6.90,"Marque : EleyEtat de l'objet : NeufType : StandardType d'ogive : Creuse 22LR ELEY HIGH VELOCITY HP UNIQUEMENT22 Long Rifle, haute vélocité, pointe creuse, douille noire oxydée.10 boîtes de 50 soit un lot de 500.",113.38,[],[50]
856,50 CARTOUCHES RWS RIFLE MATCH S CALIBRE 22LR,https://www.naturabuy.fr/50-CARTOUCHES-RWS-RIFLE-MATCH-S-CALIBRE-22LR-item-10149821.html,RWS,1,8.79,10.99,"Marque : RWSEtat de l'objet : NeufType : MatchType d'ogive : Plomb Article de catégorie C VENTE INTERDITE AUX MINEURS.Liste des documents nécessaires à l'achat : Une copie de votre pièce d'identité Une copie de votre licence de tir en cours validée par le médecin OU Une copie de votre pièce d'identité Une copie de votre permis de chasser Une copie de votre validation de permis de la saison en cours ou précédentes La munition RWS Rifle Match S (comme supersonique) atteint une vitesse élevée qui garantit un tir tendu jusqu'à 50 et 100 mètres. Compatible avec toutes les armes en .22 LR, elle présente toutefois les meilleures aptitudes aux épreuves de carabine : 50 mètres, 100 mètres ou encore biathlon. Elle constituera enfin une très bonne munition d'entraînement pour les utilisateurs de RWS R100. La cartouche RWS Rifle Match S est pourvue d'une ogive plomb type round nose d'un poids de 40 grains. La balle sort du canon à une vitesse de 345 m/s avec une énergie initiale de 155 Joules....",19.78,[50],"[50, 50, 50]"
881,( Boite de 500 cartouches)Cartouches 22 LR Solid Point Aguila,https://www.naturabuy.fr/-Boite-500-cartouches-Cartouches-22-LR-Solid-Point-Aguila-Vitesse-standard-balle-pleine--item-5234890.html,Aguila,1,90.02,6.90,Marque : AguilaEtat de l'objet : NeufType : StandardType d'ogive : Plomb BOITE DE 500 CARTOUCHES UNIQUEMENTOgive pleine en plomb à profil Round Nose. Vitesse : 1130 fps (344 m/s) soit de 153 joules.10 boîtes de 50 soit 1 carton de 500.,96.92,[],[50]


### new way 3

In [12]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ 1st

def find_ammo_quantity(text):
    pattern = r'\b(?:50|[35]25)\b'  # Regular expression pattern to match 50, 525, or 325
    numbers = [int(match) for match in re.findall(pattern, text)]
    return numbers[0] if numbers else 0

def compare_first_numbers(num1, num2):
    return int(num1 == num2)

def solution_a(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_ammo_quantity)
    df['ammo_qtty_description'] = df['product_description'].apply(find_ammo_quantity)
    df['qtty_titledescr_equal'] = df.apply(lambda row: compare_first_numbers(row['ammo_qtty_title'], row['ammo_qtty_description']), axis=1)
    return df

# Usage example:
df_v11 = solution_a(df)

In [13]:
df_v11.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_titledescr_equal
1137,Cartouches 22 LR Aguila Subsonic Hollow Point,https://www.naturabuy.fr/Cartouches-22-LR-Aguila-Subsonic-Hollow-Point-item-7831722.html,Aguila,1,118.00,14.90,Marque : AguilaEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Ogive 38 grains subsonique à tête creuse. Cartouche vélocité standard : 1024 fps (312 m/s) soit une énergie de 120 joules. 10 boîtes de 50 soit 1 carton de 500.Détails techniques : Législation pour la vente : Catégorie C - Présentation permis de chasse + validation OU licence de tir OU licence de ball-trap (Plus de détails dans législation)Calibre : .22 LRCdt : 500Énergie (J) : 120Grains : 38Grammes : 2.46Vitesse (m/s) : 312,132.90,0,50,0
360,Balles RWS R100 - Cal. 22LR - 22LR / Par 1 / 40,https://www.naturabuy.fr/Balles-RWS-R100-Cal-22LR-Par-1-item-9179470.html,RWS,1,25.00,9.90,"Marque : RWSEtat de l'objet : NeufType : StandardType d'ogive : Plomb Munitions RWS Cal.22lr Premium Line R100La munition RWS R 100 est la préférée des tireurs sportifs de compétition au niveau international. Vitesse supersonique et précision remarquable font de la RWS R 100 une munition de premier choix pour le tir sur silhouettes à l'arme d'épaule et de poing.Adaptée au tir à 50 et 100 m.Merci de vous renseigner auprès de vos instances quant à l'utilisation de munitions à percussion annulaire RWS .22lr R100 en biathlon.Le poids de la balle est exactement de 2.675 +/- 0.005 grammes Quantité: Par 1, par 10 ou par 20 (voir options)",34.90,0,50,0
287,Balles Federal Premium Match - Cal. 22 LR - Par 1 / 22LR / 40,https://www.naturabuy.fr/Balles-Federal-Premium-Match-Cal-22-LR-Par-1-item-9207325.html,Federal,1,31.25,9.90,"Marque : FederalEtat de l'objet : NeufType : MatchType d'ogive : Plomb Utilisez le meilleur pour la compétition. Ce sont les chargements d'usine les plus précis. Et c'est la technologie dont vous avez besoin pour ramener quelques médailles à la maison. /Caractéristiques: Calibre 22 LRBoîte50OgiveSolid100ms277Poids2.59Joules 100m100Joules 50m117Grains40 Quantité: Par 1, par 10 ou par 20 (voir options)",41.15,0,0,1
608,CCI STINGER,https://www.naturabuy.fr/CCI-STINGER-item-10013678.html,CCI,1,15.00,9.00,Marque : CCIEtat de l'objet : NeufType d'ogive : Creuse MUNITIONS CCI HAUTE VITESSE AVEC BALLE DE 32 GRS A 499 M-S MUNITIONS DITES DE VARMINT TRES HAUT POUVOIR PENETRANT ET EXPENSIF A MON AVIS LA MUNITION 22LR LA PLU PUISSANTE DU MARCHE,24.00,0,0,1
21,RWS 22 LR High Velocity Green 1.6g 24gr x10 boites,https://www.naturabuy.fr/RWS-22-LR-High-Velocity-Green-1-6g-24gr-x10-boites-item-9861604.html,RWS,1,134.00,0.00,Marque : RWSEtat de l'objet : Neuf !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR High Velocity Green de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR High Velocity Green Ogive : Zinc Poids : 1.6 g Grains : 24 gr Munition / boite : 50 NEUF X10 BOITES Frais de port offert ! profitez-en ! ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,134.00,0,50,0
232,Balles Norma RIMF CTG Xtreme - Cal. 22 LR - 2.8 g,https://www.naturabuy.fr/Balles-Norma-RIMF-CTG-Xtreme-Cal-22-LR-2-8-g-item-10076550.html,Norma,1,20.40,9.90,"Marque : NormaEtat de l'objet : NeufType : StandardType d'ogive : Plomb Caractéristiques : - Spécialement développée pour le tir longue distance (>200m)- Conception unique avec coeff. balistique élevé- Trajectoire tendue- Poids élevé pour une meilleure trajectoire- Recommandé pour les concours de tir longue distance en percussion annulaire- Calibre : .22 long pour carabine- Projectile de plomb de 2,8g, V0 355 m/s (longueur de canon : 65 cm)- Boite de 50",30.30,0,50,0
428,Balles RWS Subsonic Hp - Cal. 22LR - Par 20,https://www.naturabuy.fr/Balles-RWS-Subsonic-Hp-Cal-22LR-Par-20-item-9273642.html,RWS,1,182.00,9.90,"Marque : 

### new way 4

In [20]:
def find_matching_numbers(text):
    pattern = r'\b(\d+)\b'
    numbers = re.findall(pattern, text)
    matching_numbers = [int(num) for num in numbers if int(num) % 50 == 0 or int(num) in [325, 525]]
    return matching_numbers

def solution_a(df):
    df['ammo_qtty_title'] = df['product_name'].apply(lambda x: find_matching_numbers(x))
    df['ammo_qtty_description'] = df['product_description'].apply(lambda x: find_matching_numbers(x))

    # Find the first match in each column and store in new column 'qtty_equal'
    df['qtty_equal'] = (df['ammo_qtty_title'].apply(lambda x: x[0] if len(x) > 0 else 0) ==
                        df['ammo_qtty_description'].apply(lambda x: x[0] if len(x) > 0 else 0)).astype(int)

    # Fill any missing values with 0
    df['ammo_qtty_title'] = df['ammo_qtty_title'].apply(lambda x: x[0] if len(x) > 0 else 0)
    df['ammo_qtty_description'] = df['ammo_qtty_description'].apply(lambda x: x[0] if len(x) > 0 else 0)

    return df

def solution_b(df):
    df['ammo_qtty_title'] = df['product_name'].apply(lambda x: find_matching_numbers(x))
    df['ammo_qtty_description'] = df['product_description'].apply(lambda x: find_matching_numbers(x))

    # New columns to check if all numbers are equal in 'ammo_qtty_title' and 'ammo_qtty_description'
    df['qtty_title_equal'] = df['ammo_qtty_title'].apply(lambda x: int(len(set(x)) == 1))  # 1 if all numbers are equal, 0 otherwise
    df['qtty_titledescr_equal'] = df['ammo_qtty_description'].apply(lambda x: int(len(set(x)) == 1))  # 1 if all numbers are equal, 0 otherwise

    return df

In [21]:
# Apply Solution A and Solution B
df_solution_a = solution_a(df.copy())
df_solution_b = solution_b(df.copy())

In [25]:
df.head()

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4


In [23]:
df_solution_a.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_equal
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9,325,325,1
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4,0,0,1
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9,325,325,1
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4,0,0,1
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4,0,0,1
5,Boîte 22 LR ancienne NCS 30,https://www.naturabuy.fr/Boite-22-LR-ancienne-NCS-30-item-10653948.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, fabrication Nouvelle Cartoucherie de Survilliers (Cartouche rie française). Bôite complète en en parfait état. Envoie par mondial relais 4,40 €",24.4,0,0,1
6,Boîte ancienne 22 LR Fiocchi ultrasonic,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-ultrasonic-item-10653942.html,Fiocchi,0,15.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : Standard Longue - haute vitesseType d'ogive : Cuivre Boîte ancienne Fiocchi Ultrasonic, boîte en parfait état. Envoie par mondial relais 4 € 40",19.4,0,0,1
7,Boite ancienne T22,https://www.naturabuy.fr/Boite-ancienne-T22-item-10270875.html,Winchester,0,10.0,4.4,Marque : WinchesterEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte complète de Winchester T22. Dernier exemplaire Envoi par Mondial Relais 4€40,14.4,0,0,1
8,MUNITIONS SUBSONIC CAL. 22 LR boite de 50 balles,https://www.naturabuy.fr/MUNITIONS-SUBSONIC-CAL-22-LR-item-9413972.html,Winchester,1,18.9,8.6,Marque : WinchesterEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Ogives en plomb 42 grains. Profil Hollow Point (Pointe creuse) expansive,27.5,50,0,0
9,22 lr bosquette x 50 en vrac conique,https://www.naturabuy.fr/22-lr-bosquette-100-vrac-conique-item-10385603.html,NCS,1,1.0,9.0,Marque : NCSEtat de l'obj

In [26]:
df_solution_b.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_title_equal,qtty_titledescr_equal
219,Cartouches 22LR RWS PISTOL MATCH par 500,https://www.naturabuy.fr/Cartouches-22LR-RWS-PISTOL-MATCH-par-500-item-10709133.html,RWS,1,86.0,14.00,Marque : RWSEtat de l'objet : NeufType : MatchType d'ogive : Plomb Cartouches 22LR RWS PISTOL MATCH -par 500 !!!,100.00,[500],[500],1,1
238,Balles Winchester T22 - Cal. 22LR - 40 gr LRN - 22LR / Par 1 / 40,https://www.naturabuy.fr/Balles-Winchester-T22-Cal-22LR-40-gr-LRN-Par-1-item-9207515.html,Winchester,1,7.5,9.90,"Marque : WinchesterEtat de l'objet : NeufType : StandardType d'ogive : Plomb - Boîte de 50 balles- Calibre : 22LR- Ogive : Target Quantité: Par 1, par 10 ou par 20 (voir options) Calibre22LRGrains40Vitesse à 0 m331 m/s L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et validés.",17.40,[],"[50, 0]",0,0
835,STINGER - CCI,https://www.naturabuy.fr/STINGER-CCI-item-7316795.html,CCI,1,20.5,30.12,"Marque : CCIEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse Type: CARTOUCHE - Marque: CCI - Modèle: STINGER - Calibre: .22 LR - Type de balle: CPHP - Ogive: Pointe creuse cuivrée - Poids grain (GR): 32 GR - Poids (g): 2,07 g - V0 (m/s): 500 - V50 (m/s): 394 - V100 (m/s): 325 - E0 (Joules): 259 - E50 (Joules): 161 - E100 (Joules): 110 - Conditionnement: BOÎTE DE 50 - Catégorie : Catégorie C",50.62,[],"[500, 325, 50]",0,0
607,".22 court, Target (Calibre: .22 court)",https://www.naturabuy.fr/-22-court-Target-Calibre-22-court--item-9185512.html,Inconnue,1,7.4,14.90,"Marque : InconnueEtat de l'objet : NeufType : StandardType d'ogive : Plomb Cartouche à percussion annulaire, .22 courte, target. Par boite = 50 cartouches. Prix quantitatif pour les munitions Sellier & Bellot sur demande.",22.30,[],[50],0,1
918,22lr RWS Biathlon Special Match,https://www.naturabuy.fr/22lr-RWS-Biathlon-Special-Match-item-8769332.html,Norma,1,15.0,4.95,"Marque : NormaEtat de l'objet : NeufType : MatchType d'ogive : Plomb Boite de 50 cartouches 22LR Marque RWS Biathlon Special Match Ogives : 40 grains (2.6grammes) , plomb , round nose Vitesse: 1050 fps (320 m/ s) Energie : 137 joules Utilisation : biathlon , jusqu'à -20°C en stock",19.95,[],"[50, 1050]",0,0
100,RWS 22 LR R50 2.6g 40gr x1 boite,https://www.naturabuy.fr/RWS-22-LR-R50-2-6g-40gr-x1-boite-item-9861174.html,RWS,1,26.8,8.00,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR R50 de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR R50 Ogive : plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X1 BOITE ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,34.80,[],[50],0,1
624,BOITE DE 100 CARTOUCHES CCI 22 LR GRENNTAG PRECISION HAUTE VITESSE COMPETITION 1070 FPS 40 GR,https://www.naturabuy.fr/BOITE-DE-100-CARTOUCHES-CCI-22-LR-GRENNTAG-PRECISION-HAUTE-VITESSE-COMPETITION-1070-FPS-40-GR-item-9727987.html,CCI,1,22.0,6.50,"Marque : CCIEtat de l'objet : NeufType : Match Bonjour, Je vends une boîte neuve de 100 cartouches CCI MATCH Haute Vitesse 1070 FPS GRENNTAG PRECISION 40 Grains 100 Cartouches 22LR LEAD ROUND NOSE Compétition Haute Vitesse 1070 FPS Cordialement",28.50,[100],"[100, 100]",1,1
40,RWS 22 LR Target Pistol 2.6g 40gr x1 boite,https://www.naturabuy.fr/RWS-22-LR-Target-Pistol-2-6g-40gr-x1-boite-item-9861417.html,RWS,1,8.9,8.00,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR Target Pistol de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR Target Pistol Ogive : Plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X1 BOITE ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96

### old solution 1

In [ ]:
## regex to catch any number divisible by 50 (min qtty of rounds in a box of ammo)
#def extract_bullet_qtty(text):
#    # match any number that is divisible by 50 without remainder
#    regex = r"\b(0|[5-9]\d*[0]|100)\s*(?:boites de\s*)?(?:cartouches|balles|munitions)\b"
#    match = re.search(regex, text, re.IGNORECASE)
#    if match:
#        # extract the matched number and convert it to integer
#        qtty = int(match.group(1))
#        # round the quantity to the nearest 50
#        qtty = (qtty // 50) * 50
#        return qtty
#    else:
#        return None
#
## check titles with regex
#df['bullet_qtty'] = df['product_name'].apply(extract_bullet_qtty)
#
## check for empty cells, if any do a pass of regex on product description --- !!! DUPE avoidance !!!
#df.loc[df['bullet_qtty'].isnull(), 'bullet_qtty'] = df['product_description'].apply(extract_bullet_qtty)
#
## if still no data, we fill with 50 for default min number of ammo per box
#df['bullet_qtty'].fillna(50, inplace=True)

In [ ]:
# qtty of ammo search in TITLE

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern1 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_title(df):
    # Find all matching numbers and store them in a list
    matches = df['product_name'].apply(lambda x: re.findall(pattern1, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_title_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [ ]:
find_ammo_qtty_title(df)

In [ ]:
df_new.head(3)

In [ ]:
# qtty of ammo search in DESCRIPTION

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern2 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_description(df):
    # Find all matching numbers and store them in a list
    matches = df['product_description'].apply(lambda x: re.findall(pattern2, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_description_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [ ]:
df_new2 = find_ammo_qtty_description(df)
df_new2.head(3)

In [ ]:
# qtty of boxes search

# Regular expression pattern for finding numbers before/after French words that mean "box" or "boxes"
pattern3 = r'\b(\d+)\s*(?:(?:LOT|PAQUET)\s+DE\s+)?(?:(\d+)\s*(?:BOITE|BOITES))\b|\b(?:PAR\s+(\d+))\b'

# Function to find matching numbers and store in new columns
def find_box_numbers(df):
    # Find all matching numbers and store them in a list
    matches_name = df['product_name'].apply(lambda x: re.findall(pattern3, str(x)))
    matches_desc = df['product_description'].apply(lambda x: re.findall(pattern3, str(x)))
    matches = matches_name + matches_desc
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'box_qty_{j+1}'
            df.at[i, column_name] = ''.join(match[j])
    
    return df

In [ ]:
df_new3 = find_box_numbers(df)
df_new3.head(3)

In [ ]:
df.head()

In [ ]:
# V2 for bullet quantities

# col1 for qtty from title and description as a flat number
# col2 for num of boxes * 50
# if both are None we set default number of 50 in col3

def extract_bullet_qtty(text):
    # Check 1: total amount of bullets
    regex1 = r"\b(\d+)\s*boîte(?:s)?\s*de\s*(50|525)\s*(?:cartouches|balles|munitions)\b|\bMunition \/ boite\s*:\s*(\d+)\s*x\s*(50|525)\b|\bQuantité\s*:\s*(50|525)\b"
    match1 = re.search(regex1, text, re.IGNORECASE)
    if match1:
        qtty = int(match1.group(1)) if match1.group(1) is not None else int(match1.group(2))
        return qtty
    # Check 2: amount of boxes
        regex2 = r"\bLOT DE (\d+) BOITES\b"
    match2 = re.search(regex2, text, re.IGNORECASE)
    if match2:
        qtty = int(match2.group(1)) * 50
        return qtty
    else:
        return 0

# create new columns for ammo quantity
df["bullet_qtty_v1"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) or extract_bullet_qtty(x["product_description"]) or 0)
df["bullet_qtty_v2"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_name"]) else extract_bullet_qtty(x["product_description"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_description"]) else 0, axis=1)

# set default value of 50 for products with missing values
df["bullet_qtty_default"] = df.apply(lambda x: 50 if x["bullet_qtty_v1"] == 0 and x["bullet_qtty_v2"] == 0 else None, axis=1)

df.head(5)

In [ ]:
# Function to extract numbers divisible by 50 from a given text
def extract_divisible_by_50(text):
    numbers = re.findall(r'\b(?:[5-9][0]|50)\b', text)
    return [int(num) for num in numbers]

# Apply the function to "product_name" and "product_description" columns and store the result in "ammo_qtty" column
df['ammo_qtty'] = df['product_name'].apply(extract_divisible_by_50) + df['product_description'].apply(extract_divisible_by_50)

# Filter and keep only the numbers divisible by 50 without remainders
df['ammo_qtty'] = df['ammo_qtty'].apply(lambda x: [num for num in x if num % 50 == 0])

In [ ]:
df.ammo_qtty[:10]

In [ ]:
df.columns